In [11]:
import pandas as pd
import numpy as np
import gradio as gr
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import warnings
warnings.filterwarnings("ignore")

np.random.seed(42)

In [ ]:
DATASET_PATH = "student_performance.csv"
DF = pd.read_csv(DATASET_PATH)

In [13]:
LE_PARTICIPATION = LabelEncoder()
LE_STATUS = LabelEncoder()

DF["PARTICIPATION_ENCODED"] = LE_PARTICIPATION.fit_transform(DF["participation"])
DF["STATUS_ENCODED"] = LE_STATUS.fit_transform(DF["final_status"])

In [14]:
FEATURES = [
    "attendance_percent",
    "assessment_score",
    "late_submissions",
    "PARTICIPATION_ENCODED"
]

TARGET = "STATUS_ENCODED"
CLASS_NAMES = LE_STATUS.classes_

X = DF[FEATURES].values
y = DF[TARGET].values

In [15]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [16]:
SCALER = StandardScaler()
X_train_scaled = SCALER.fit_transform(X_train)
X_test_scaled = SCALER.transform(X_test)

In [17]:
MODEL = RandomForestClassifier(n_estimators=200, max_depth=10, random_state=42)
MODEL.fit(X_train_scaled, y_train)

RandomForestClassifier(max_depth=10, n_estimators=200, random_state=42)

In [18]:
y_pred = MODEL.predict(X_test_scaled)
TEST_ACCURACY = accuracy_score(y_test, y_pred)

print("Accuracy:", TEST_ACCURACY)
print(classification_report(y_test, y_pred, target_names=CLASS_NAMES))

Accuracy: 1.0
              precision    recall  f1-score   support

     At Risk       1.00      1.00      1.00         5
        Fail       1.00      1.00      1.00         4
        Pass       1.00      1.00      1.00         8

    accuracy                           1.00        17
   macro avg       1.00      1.00      1.00        17
weighted avg       1.00      1.00      1.00        17



In [19]:
def predict_student_performance(attendance, assessment, late_subs, participation):
    participation_encoded = LE_PARTICIPATION.transform([participation])[0]

    input_data = np.array([
        [attendance, assessment, late_subs, participation_encoded]
    ])

    input_scaled = SCALER.transform(input_data)

    prediction = MODEL.predict(input_scaled)[0]
    probabilities = MODEL.predict_proba(input_scaled)[0]

    status = CLASS_NAMES[prediction]
    confidence = probabilities[prediction] * 100

    if status == "Pass":
        color = "#16a34a"
        message = "The student is performing well academically."
    elif status == "At Risk":
        color = "#ca8a04"
        message = "The student may need academic support."
    else:
        color = "#dc2626"
        message = "The student is at risk of failing."

    return f"""
### Student Performance Result

**Status:** <span style="color:{color}; font-size:18px"><b>{status}</b></span>  
**Confidence:** {confidence:.1f}%

{message}
"""

In [20]:
with gr.Blocks(theme=gr.themes.Soft(), title="Student Performance Dashboard") as app:

    gr.Markdown("# Student Academic Performance Dashboard")
    gr.Markdown(
        "Enter student academic details to assess overall performance."
    )

    with gr.Group():
        attendance_input = gr.Slider(50, 100, 80, label="Attendance (%)")
        assessment_input = gr.Slider(40, 100, 75, label="Assessment Score (%)")
        late_subs_input = gr.Slider(0, 5, 1, label="Late Submissions")
        participation_input = gr.Radio(
            ["Low", "Medium", "High"],
            label="Class Participation",
            value="Medium"
        )

        predict_btn = gr.Button("Analyze Performance", variant="primary")

    result_output = gr.Markdown()

    predict_btn.click(
        fn=predict_student_performance,
        inputs=[
            attendance_input,
            assessment_input,
            late_subs_input,
            participation_input
        ],
        outputs=result_output
    )

    gr.Markdown("---")
    gr.Markdown(f"Model Accuracy: {TEST_ACCURACY*100:.1f}%")

app.launch(share=True)

* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://fea08d09c7d0c93158.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
